In [3]:
import pandas as pd
# import pubchempy as pcp
import numpy as np
from rdkit import Chem 
from rdkit import RDLogger  
from rdkit.Chem import PandasTools, SaltRemover

data = pd.DataFrame()

# Data import
Here i import the data from the data forms they were taken as and format them similar to each other

## Hansen
Hansen dataset taken from: https://pubs.acs.org/doi/10.1021/ci900161g
Comprising 6512 molecules from CCRIS, Helma et al., Kazius et al., Feng et al., VITIC, and GeneTox databases
Given is the SMILES, CAS No., and Ames mutagenicty
Postivite Ames mutagenicity is defined as having a positive Ames result for any salmonella strain for either with or without S9. Based on this it is said to be more accurate as this result will not change with additional testing.

In [4]:
path_to_hansen = '/home/samuel/honours_redo/1-data/datasets/Hansen_et_al_2009.csv'

f = pd.read_csv(path_to_hansen, names=["smiles","CASRN", "Ames"])
data =  data.append(f[["smiles",'CASRN',"Ames"]])
hansen_data = data

FileNotFoundError: [Errno 2] No such file or directory: '/home/samuel/honours_redo/1-data/datasets/Hansen_et_al_2009.csv'

## Bruschweiler
Brushweiler dataset taken from: https://www.sciencedirect.com/science/article/pii/S0273230017301812 
Dataset made up of 23 azo dye cleavage products. Given is the CAS No, SMILES, and Ames results for S9 +/- for strain TA98 and then TA100

In [ ]:
path_to_bruschweiler = "/home/samuel/honours_redo/1-data/datasets/Bruschweiler2017_Ames_Source.csv"

f = pd.read_csv(path_to_bruschweiler).drop(['TA98-S9','TA98+S9','TA100-S9','TA100+S9'],axis=1)
f.rename(columns={'SMILES':'smiles','Overall':'Ames'},inplace=True)
data =  data.append(f[["smiles",'CASRN',"Ames"]])

<ipython-input-14-1674e8c8027c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data =  data.append(f[["smiles",'CASRN',"Ames"]])


## Ates
Ates Dataset taken from: https://doi.org/10.1093/mutage/gew008132 132 cosmetic substances taken from publicaly available information at: http://ec.europa.eu/health/scientific_committees/consumer_safety/opinions/index_en.htm Results are defined the same as in Hansen but specify that the 5 reccomended strains are tested for both S9 + and -.

In [ ]:
path_to_Ates = '/home/samuel/honours_redo/1-data/datasets/Ates_et_at_2016.csv'

f = pd.read_csv(path_to_Ates,skiprows=[0,1])
f.rename(columns={'Cas No':'CASRN','Result Ames test':'Ames'},inplace=True)
f["smiles"] = "not available"
f["Ames"] = f['Ames'].apply(lambda x: 1 if x=='POS' else 0)
data =  data.append(f[["smiles",'CASRN',"Ames"]])

<ipython-input-15-9d1246336015>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data =  data.append(f[["smiles",'CASRN',"Ames"]])


## OECD data
taken from the OECD toolbox using the databases available.

In [ ]:
def converter(value):
    if str(value) in ['Positive']:
        return 1
    elif value in ['Negative']:
        return 0
    else:
        return np.nan

## Adding OECD data
directory = '/home/samuel/honours_redo/1-data/datasets/OECD_inv_data/woke'
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        df = pd.read_csv(os.path.join(directory, filename), encoding='utf16')
        df["smiles"] = df["SMILES"]
        df['CASRN'] = df['CAS Number']
        if "OVERALL" in df.columns:
            df = df[pd.notna(df['OVERALL']) & pd.notna(df['SMILES'])]
            df["Ames"] = df['OVERALL'].apply(lambda x: converter(x))

            if len(df[pd.notna(df['Ames'])]) == 0:
                df = df[pd.notna(df['Value.MeanValue']) & pd.notna(df['SMILES'])]
                df["Ames"] = df['Value.MeanValue'].apply(lambda x: converter(x))

        else:
            df = df[pd.notna(df['Value.MeanValue']) & pd.notna(df['SMILES'])]
            df["Ames"] = df['Value.MeanValue'].apply(lambda x: converter(x))
        df = df[pd.notna(df['Ames'])]
        df["Ames"] = df["Ames"].astype(int)
        data = data.append(df[["smiles", "Ames"]])
        print(filename, "DONE")
print(len(data))

<ipython-input-16-853496e9464a>:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename), encoding='utf16')
<ipython-input-16-853496e9464a>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df['Value.MeanValue'].apply(lambda x: converter(x))
<ipython-input-16-853496e9464a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df["Ames"].astype(int)
<ipython-input-16-853496e9464a>:29: Fut

NICNAS.csv DONE


<ipython-input-16-853496e9464a>:13: DtypeWarning: Columns (0,58,59,60,61,62,63,64,65,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename), encoding='utf16')
<ipython-input-16-853496e9464a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df['OVERALL'].apply(lambda x: converter(x))
<ipython-input-16-853496e9464a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df["Ames"].astype(int)
<ipython-input-16

DSSTOX.csv DONE


<ipython-input-16-853496e9464a>:13: DtypeWarning: Columns (0,58,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename), encoding='utf16')
<ipython-input-16-853496e9464a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df['OVERALL'].apply(lambda x: converter(x))
<ipython-input-16-853496e9464a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df["Ames"].astype(int)
<ipython-input-16-853496e9464a>:29: Fu

...

<ipython-input-16-853496e9464a>:13: DtypeWarning: Columns (0,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename), encoding='utf16')
<ipython-input-16-853496e9464a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df['OVERALL'].apply(lambda x: converter(x))
<ipython-input-16-853496e9464a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df["Ames"].astype(int)
<ipython-input-16-853496e9464a>:29:

ECHA_PR.csv DONE
HPVC_OECD.csv DONE


<ipython-input-16-853496e9464a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df['OVERALL'].apply(lambda x: converter(x))
<ipython-input-16-853496e9464a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df["Ames"].astype(int)
<ipython-input-16-853496e9464a>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df[["smiles", "Ames"]])
<ipython-input-16-853496e9464a>:13: Dtype

TSCA.csv DONE
COSING.csv DONE
Canada_DSL.csv DONE
146211


<ipython-input-16-853496e9464a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df['OVERALL'].apply(lambda x: converter(x))
<ipython-input-16-853496e9464a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ames"] = df["Ames"].astype(int)
<ipython-input-16-853496e9464a>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df[["smiles", "Ames"]])


# Data manipulation

In [ ]:
def format_data(df):    
    def check_Ames(df):
        df = df[df['Ames'].isin([1,0])]
        return df

    def find_can_smi(df):
        def try_can_smi(x):
            if x == 'BrCc1ccc(Br)cc1' :print('BrCc1ccc(Br)cc1')
            try:
                return Chem.MolToSmiles(Chem.MolFromSmiles(x))
            except:
                print('Molecule lost:',x)
                return "not found"
        df["smiles"] = df["smiles"].apply(lambda x: try_can_smi(x))
        df = df.drop_duplicates(subset=['smiles'])
        df = df[df['smiles'] != "not found"]
        return df
    def clean_molecules(df):
        def remove_salts(smiles):
            if not smiles == np.nan:
                remover = SaltRemover.SaltRemover()
                mol = Chem.MolFromSmiles(smiles)
                if not mol or not smiles:
                    print(smiles)
                    return np.nan
                
                res = remover.StripMol(mol)
                if Chem.MolToSmiles(res):
                    return Chem.MolToSmiles(res)
                else:
                    print("Molecule salt removal removed whole molecule", smiles, "Molecule was retained but please check")
                    return smiles
            else:
                print(smiles)
                return np.nan
        # def remove_solutions(smiles):
            ####
        df['smiles'] = df['smiles'].apply(lambda x: remove_salts(x))
        # df['smiles'] = df['smiles'].apply(lambda x: remove_solutions(x))
        return df
    export_df = df.copy()
    export_df = check_Ames(export_df)
    export_df = find_can_smi(export_df)
    export_df = clean_molecules(export_df)
    return export_df

def only_aromatic_amines(df):
    def get_mol(row):
        try:
            mol = Chem.MolFromSmiles(row['smiles'])
            test = Chem.MolToSmiles(mol)
            return mol
        except:
            try:
                mol = Chem.MolFromSmiles(pcp.get_compounds(row['CASRN'], 'name')[0].canonical_smiles)
                test = Chem.MolToSmiles(mol)
                return mol
            except:
                print("error on:",row['smiles'],row['CASRN'])
                return "Not found"
    def AA_check(row):
        m = get_mol(row)
        if m != "Not found":
            for a in m.GetAtoms():
                if a.GetAtomicNum() == 7 and any([n.GetIsAromatic() for n in a.GetNeighbors()]) and not a.IsInRing()  and (a.GetFormalCharge() >= 0) and len([n.GetIsAromatic() for n in a.GetNeighbors()]) ==1:     ## Checks for a nitrogen and then if the neighbors of the nitrogen are aromatic and also checks that it isn't cationic
                    return 1
                else: 
                    return 0
    export_df = df.copy()
    export_df["is AA"] = export_df.apply(lambda row: AA_check(row),axis=1)
    export_df = export_df[export_df["is AA"] == 1]
    return export_df

## Hansen all molecules

In [ ]:
# I disable error logs as i account for them generally in my get mol function by searching for a better smiles using the CAS number. Please use for debugging if needed
RDLogger.DisableLog('rdApp.*') 
data_final = format_data(hansen_data)
a = data_final
print("original total:", len(hansen_data))
print('final total:', len(data_final))
print("original Ames split:", "yes="+str(len(hansen_data[hansen_data["Ames"] == 1])), "no="+str(len(hansen_data[hansen_data["Ames"] == 0])))
print("final Ames split:", "yes="+str(len(data_final[data_final["Ames"] == 1])), "no="+str(len(data_final[data_final["Ames"] == 0])))
data_final = data_final[["smiles",'Ames']].reset_index(drop = True)
b = data_final
##### saving data in two forms
    # csv form
data_final.to_csv('data/raw/hansen_raw/Hansen_all_mols.csv', index=False)
    # sdf form
PandasTools.AddMoleculeColumnToFrame(data_final,'smiles','Molecule')
data_final['idx'] = data_final.index
Chem.PandasTools.WriteSDF(data_final, "data/raw/hansen_raw/Hansen_all_mols.sdf", molColName='Molecule', properties=list(data_final.columns), idName = "idx")

Molecule lost: NNC(=O)CNC(=O)\C=N\#N 
Molecule lost: O=C1NC(=O)\C(=N/#N)\C=N1 
Molecule lost: NC(=O)CNC(=O)\C=N\#N 
Molecule lost: CCCCN(CC(O)C1=C\C(=N/#N)\C(=O)C=C1)N=O 
Molecule lost: NC(COC(=O)\C=N/#N)C(=O)O 
Molecule lost: CCN(CC(O)C1=CC(=O)\C(=N\#N)\C=C1)N=O 
Molecule salt removal removed whole molecule CS(=O)(=O)O Molecule was retained but please check
Molecule salt removal removed whole molecule O=C(O)[C@@H](O)[C@H](O)C(=O)O Molecule was retained but please check
Molecule salt removal removed whole molecule CC(=O)O Molecule was retained but please check
Molecule salt removal removed whole molecule O=C(O)C(=O)O Molecule was retained but please check
Molecule salt removal removed whole molecule O=C(O)/C=C\C(=O)O Molecule was retained but please check
Molecule salt removal removed whole molecule C1CCC(NC2CCCCC2)CC1 Molecule was retained but please check
original total: 6512
final total: 6505
original Ames split: yes=3503 no=3009
final Ames split: yes=3496 no=3009


## All data Aromatic amines

In [ ]:
# I disable error logs as i account for them generally in my get mol function by searching for a better smiles using the CAS number. Please use for debugging if needed
RDLogger.DisableLog('rdApp.*') 
data_final = format_data(data)
data_final = only_aromatic_amines(data_final)

print("original total:", len(data))
print('final total:', len(data_final))
print("original Ames split:", "yes="+str(len(data[data["Ames"] == 1])), "no="+str(len(data[data["Ames"] == 0])))
print("final Ames split:", "yes="+str(len(data_final[data_final["Ames"] == 1])), "no="+str(len(data_final[data_final["Ames"] == 0])))
data_final = data_final[["smiles",'Ames']].reset_index(drop = True)

##### saving data in two forms
    # csv form
data_final.to_csv("data/raw/selected_molecules.csv", index=False)
    # sdf form
PandasTools.AddMoleculeColumnToFrame(data_final,'smiles','Molecule')
data_final['idx'] = data_final.index
Chem.PandasTools.WriteSDF(data_final, "data/raw/selected_molecules.sdf", molColName='Molecule', properties=list(data_final.columns), idName = "idx")

original total: 146211
final total: 457
original Ames split: yes=38625 no=107586
final Ames split: yes=305 no=152


TESTING OTHER AROMATIC AMINE DECIDER

In [ ]:
def only_aromatic_amines_2(df):
    def get_mol(row):
        try:
            mol = Chem.MolFromSmiles(row['smiles'])
            test = Chem.MolToSmiles(mol)
            return mol
        except:
            try:
                mol = Chem.MolFromSmiles(pcp.get_compounds(row['CASRN'], 'name')[0].canonical_smiles)
                test = Chem.MolToSmiles(mol)
                return mol
            except:
                print("error on:",row['smiles'],row['CASRN'])
                return "Not found"
    def aa_check_2(row):

        m = get_mol(row)
        if m != "Not found":
            for a in m.GetAtoms():
                if a.GetAtomicNum() == 7 and any([n.GetIsAromatic() for n in a.GetNeighbors()]) and not a.IsInRing()  and (a.GetFormalCharge() >= 0) and len([n.GetIsAromatic() for n in a.GetNeighbors()]) ==1:     ## Checks for a nitrogen and then if the neighbors of the nitrogen are aromatic and also checks that it isn't cationic
                    return 1
                else: 
                    return 0


                    
    export_df = df.copy()
    export_df["is AA"] = export_df.apply(lambda row: aa_check_2(row),axis=1)
    export_df = export_df[export_df["is AA"] == 1]
    return export_df